In [ ]:
#### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
token = user_secrets.get_secret("github")

In [ ]:
!pip install git+https://{token}@github.com/MarcoGorelli/wound-ignite.git@8d8e5c6f4fbd7eaed7e6b2022e2ee7982051c7d5

In [ ]:
from precise.skatertools.m6.competition import m6_competition_entry, m6_dump
from precise.whereami import M6_EXAMPLES
import os
import time
import json
import argparse
import pandas as pd

from precise.skaters.covariance.allcovskaters import ALL_D0_SKATERS
from precise.skaters.portfoliostatic.allstaticport import  PORT
from tqdm.notebook import tqdm

# Example of creating an M6 Competition entry using random choice of cov estimation and port construction
final_candidates = pd.read_csv('/kaggle/input/find-best-from-cv/final_candidates.csv')
BEST_F = final_candidates['f'].tolist()
BEST_PORT = final_candidates['port'].tolist()
BEST_SCALING = final_candidates['scaling'].tolist()


for f, port, scaling in zip(BEST_F, BEST_PORT, BEST_SCALING):
    for last_date in pd.date_range(end=pd.Timestamp.now(), periods=1, freq='W-Fri'):
        try:
            df = m6_competition_entry(
                f=ALL_D0_SKATERS[f],
                port=PORT[port],
                last_date=last_date,
                scale_w=scaling,
                cache_path='/kaggle/input/precise-cache/'
            )
            name = f'rps_{f}_{port}_{scaling}_{last_date}.csv'
            m6_dump(df=df,file_name=name)
        except Exception as e:
            print(repr(e))
            pass